In [1]:
# Import necessary libraries
import getpass
import glob
import os
import pandas as pd
import numpy as np
import h5py
import mne

In [3]:
file1 = "D:\\Dropbox\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt1.mat"
file2 = "D:\\Dropbox\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt2.mat"

file1_data = h5py.File(file1, 'r')
file2_data = h5py.File(file2, 'r')


file1_data.keys()
file2_data.keys()

for key in file1_data.keys():
    if key in file2_data.keys():
        print(f"Key '{key}' found in both files.")
    else:
        print(f"Key '{key}' not found in file2.")

for key in file2_data.keys():
    data_1 = file1_data[key]
    data_2 = file2_data[key]
    # Now you can work with data_1 and data_2
    print(type(data_1))
    print(type(data_2))

Key 'Keyboard' found in both files.
Key 'LFP1_AON' found in both files.
Key 'LFP1_vHp' found in both files.
Key 'LFP2_AON' found in both files.
Key 'LFP2_vHp' found in both files.
Key 'LFP3_AON' found in both files.
Key 'LFP4_AON' found in both files.
Key 'Ref' found in both files.
Key 'Respirat' found in both files.
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>
<class 'h5py._hl.group.Group'>


In [4]:
# ...existing code...
import h5py
import numpy as np

def copy_attrs(src, dst):
    for k, v in src.attrs.items():
        if k not in dst.attrs:
            dst.attrs[k] = v
        else:
            # If conflict, keep both with suffixes (only if values differ)
            try:
                # Use np.array_equal if both are arrays, else use ==
                if isinstance(dst.attrs[k], np.ndarray) or isinstance(v, np.ndarray):
                    same = np.array_equal(dst.attrs[k], v)
                else:
                    same = (dst.attrs[k] == v)
            except Exception:
                same = False
            if not same:
                dst.attrs[f"{k}_file2"] = v  # assuming src is from file2 when called second

def merge_groups(g1: h5py.Group, g2: h5py.Group, gout: h5py.Group, concat_axis=0):
    keys1 = set(g1.keys())
    keys2 = set(g2.keys())
    all_keys = keys1 | keys2

    for name in all_keys:
        in1 = name in g1
        in2 = name in g2

        if in1 and in2:
            o1 = g1[name]
            o2 = g2[name]

            if isinstance(o1, h5py.Group) and isinstance(o2, h5py.Group):
                sub = gout.create_group(name)
                copy_attrs(o1, sub)
                copy_attrs(o2, sub)
                merge_groups(o1, o2, sub, concat_axis=concat_axis)

            elif isinstance(o1, h5py.Dataset) and isinstance(o2, h5py.Dataset):
                # Can we concatenate?
                compatible = (
                    o1.dtype == o2.dtype and
                    o1.ndim == o2.ndim and
                    all((i == concat_axis) or (o1.shape[i] == o2.shape[i]) for i in range(o1.ndim))
                )
                if compatible:
                    d1 = o1[...]
                    d2 = o2[...]
                    merged = np.concatenate([d1, d2], axis=concat_axis)
                    dset = gout.create_dataset(
                        name,
                        data=merged,
                        compression=o1.compression or o2.compression,
                        chunks=o1.chunks or o2.chunks
                    )
                    copy_attrs(o1, dset)
                    copy_attrs(o2, dset)
                else:
                    # Conflict: keep both versions
                    gout.copy(o1, f"{name}_file1")
                    gout.copy(o2, f"{name}_file2")
            else:
                # One is group, one is dataset: keep both
                if isinstance(o1, h5py.Group):
                    gout.copy(o1, f"{name}_file1_group")
                else:
                    gout.copy(o1, f"{name}_file1_dataset")
                if isinstance(o2, h5py.Group):
                    gout.copy(o2, f"{name}_file2_group")
                else:
                    gout.copy(o2, f"{name}_file2_dataset")

        elif in1:
            gout.copy(g1[name], name)
        else:
            gout.copy(g2[name], name)

def merge_files(file1, file2, out_file, concat_axis=0):
    with h5py.File(file1, "r") as f1, h5py.File(file2, "r") as f2, h5py.File(out_file, "w") as fout:
        copy_attrs(f1, fout)
        copy_attrs(f2, fout)
        merge_groups(f1, f2, fout, concat_axis=concat_axis)
        f1.close()
        f2.close()
        fout.close()
# Example usage (concatenate datasets along axis 0)
file1 = r"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt1.mat"
file2 = r"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt2.mat"
outf  = r"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_merged.mat"

merge_files(file1, file2, outf, concat_axis=0)
# ...existing code...

In [7]:
file1=f"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt1.mat"
file2=f"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_pt2.mat"
outf  = f"D:\\Dropbox\\CPLab\\all_data_mat_250825\\20230817_dk1_BW_context_os2_day1_merged.mat"
file1_data= h5py.File(file1, 'r')
file2_data= h5py.File(file2, 'r')
outf_data= h5py.File(outf, 'r')
channel_names = ['LFP1_AON', 'LFP2_AON', 'LFP3_AON', 'LFP4_AON', 'LFP1_vHp', 'LFP2_vHp']

for channeli in channel_names:
    file1_channel = file1_data[channeli]
    file2_channel = file2_data[channeli]
    outf_channel = outf_data[channeli]

    print(file1_channel['values'])
    print(file2_channel['values'])
    #print(outf_channel['values'])
    print(outf_channel.keys())
file1_data.close()
file2_data.close()
outf_data.close()

<HDF5 dataset "values": shape (1, 777936), type "<f8">
<HDF5 dataset "values": shape (1, 3021431), type "<f8">
<KeysViewHDF5 ['comment', 'interval', 'length', 'offset', 'scale', 'start', 'times_file1', 'times_file2', 'title', 'units', 'values_file1', 'values_file2']>
<HDF5 dataset "values": shape (1, 777936), type "<f8">
<HDF5 dataset "values": shape (1, 3021431), type "<f8">
<KeysViewHDF5 ['comment', 'interval', 'length', 'offset', 'scale', 'start', 'times_file1', 'times_file2', 'title', 'units', 'values_file1', 'values_file2']>
<HDF5 dataset "values": shape (1, 777936), type "<f8">
<HDF5 dataset "values": shape (1, 3021431), type "<f8">
<KeysViewHDF5 ['comment', 'interval', 'length', 'offset', 'scale', 'start', 'times_file1', 'times_file2', 'title', 'units', 'values_file1', 'values_file2']>
<HDF5 dataset "values": shape (1, 777936), type "<f8">
<HDF5 dataset "values": shape (1, 3021431), type "<f8">
<KeysViewHDF5 ['comment', 'interval', 'length', 'offset', 'scale', 'start', 'times_fi

In [44]:
dk1_merged = f"D:\\Dropbox\\CPLab\\dk1_parts\\20230719_dk1_nocontext_os2_day2.mat"

dk1_data= h5py.File(dk1_merged, 'r')
channels= dk1_data.keys()

for channeli in channels:
    channel_data = dk1_data[channeli]
    file1_channel = file1_data[channeli] if channeli in file1_data else None
    file2_channel = file2_data[channeli] if channeli in file2_data else None
    print(channeli)
    #print(channel_data.keys())
    if 'values' in channel_data:
        print(channel_data['values'].shape)
        print(file1_channel['values'].shape if file1_channel is not None else None)
        print(file2_channel['values'].shape if file2_channel is not None else None)
    print("\n\n")

V20230719_dk1_nocontext_os2_day2_Ch1
(1, 2276770)
None
None



V20230719_dk1_nocontext_os2_day2_Ch2
(1, 2276770)
None
None



V20230719_dk1_nocontext_os2_day2_Ch29
(1, 1139)
None
None



V20230719_dk1_nocontext_os2_day2_Ch3
(1, 2276770)
None
None



V20230719_dk1_nocontext_os2_day2_Ch31



V20230719_dk1_nocontext_os2_day2_Ch4
(1, 2276770)
None
None



V20230719_dk1_nocontext_os2_day2_Ch5
(1, 2276769)
None
None



V20230719_dk1_nocontext_os2_day2_Ch6
(1, 2276769)
None
None



V20230719_dk1_nocontext_os2_day2_Ch8
(1, 2276769)
None
None



